In [105]:
import pickle
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
import scipy.stats
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [3]:
xgb_lexical = pickle.load(open('xgB_lexical_model.sav', 'rb'))

In [4]:
rf_pm = pickle.load(open('rf_pm_model.sav', 'rb'))

In [69]:
lr_tfidf = pickle.load(open('lr_tfidf_model.sav', 'rb'))

## Lexical model chosen was XGBoost classifer which is also an ensemble boosting model

In [8]:
# loading the pipeline for features transformation
pipeline_lecixal = pickle.load(open('pipeline_model.sav', 'rb'))

In [15]:
urldata = pd.read_csv('urldata.csv', index_col='Unnamed: 0')[:61000]

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
X_test = pipeline_lecixal.transform(urldata)

In [21]:
y_predict_lexical = xgb_lexical.predict(X_test)

In [30]:
y_prob_lexical = xgb_lexical.predict_proba(X_test)[:,1]

## Performance model chosen was Random Forest classifer

In [23]:
# loading the pipeline for features transformation
pm_pipeline = pickle.load(open('pm_pipeline.sav', 'rb'))

In [26]:
pm_data = pd.read_csv('urldata_pm.csv', index_col='Unnamed: 0')

In [28]:
X_test_pm = pm_data.drop('observedTimeOrigin', axis=1)

In [29]:
X_test_pm = pm_pipeline.transform(X_test_pm)

In [32]:
y_predict_pm = rf_pm.predict(X_test_pm)

In [34]:
y_prob_pm = rf_pm.predict_proba(X_test_pm)[:,1]

## NLP model chosen was Logistic Regression Classifier

In [70]:
## load the text transformer vectorizer
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer_model.sav', 'rb'))

In [71]:
urldata_nlp = pd.read_csv('urldata_nlp.csv')

In [72]:
X_test_nlp = urldata_nlp['clean_text'][:61000]

In [73]:
X_test_nlp = tfidf_vectorizer.transform(X_test_nlp)

In [74]:
y_predict_nlp = lr_tfidf.predict(X_test_nlp)

In [75]:
y_prob_nlp = lr_tfidf.predict_proba(X_test_nlp)[:,1]

# Ensembling the result of all three models and evaluating the model.

In [77]:
#The label for the test data set
urldata = pd.read_csv('urldata_nlp.csv')
def type_encoder(x):
    if x == 'malicious':
        return 1
    elif x == 'benign':
        return 0
    else:
        return x

urldata['target'] = urldata['type'].apply(lambda x: type_encoder(x))

y = urldata['target'][:61000]


In [99]:
y_predict_arr = [np.array(y_predict_lexical), np.array(y_predict_pm), np.array(y_predict_nlp)]
y_predict = scipy.stats.mode(np.stack(y_predict_arr), axis=0)

In [104]:
print(classification_report(y, y_predict.mode.tolist()[0]))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     45169
           1       1.00      0.97      0.98     15831

    accuracy                           0.99     61000
   macro avg       0.99      0.98      0.99     61000
weighted avg       0.99      0.99      0.99     61000



In [106]:
y_prob_arr = [np.array(y_prob_lexical), np.array(y_prob_pm), np.array(y_prob_nlp)]
y_prob = scipy.stats.mode(np.stack(y_prob_arr), axis=0)

In [107]:
fpr, tpr, thresholds = roc_curve(y, y_prob.mode.tolist()[0])
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

AUC: 0.9884041465738733
